In [ ]:
import configparser
import pandas as pd
import re
# import and instantiate TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
# import and instantiate the Logistic Regression model
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

config = configparser.ConfigParser()
config.read('./config.cfg')
train_df = pd.read_csv(config['FILES']['TRAIN'])
test_df = pd.read_csv(config['FILES']['TEST'])
print(test_df.shape)
print(test_df['id'][0:5])

test_label = pd.read_csv(config['FILES']['LABEL'])
print(test_label.shape)
print(test_label['id'][0:5])

cols_name = ['obscene', 'insult', 'toxic', 'severe_toxic', 'identity_hate', 'threat']





(153164, 2)
0    00001cee341fdb12
1    0000247867823ef7
2    00013b17ad220c46
3    00017563c3f7919a
4    00017695ad8997eb
Name: id, dtype: object
(153164, 7)
0    00001cee341fdb12
1    0000247867823ef7
2    00013b17ad220c46
3    00017563c3f7919a
4    00017695ad8997eb
Name: id, dtype: object


In [17]:
def clean_text(text):
    text = text.lower()
    for change in [(r"what's", "what is "), (r"\'s", " "), (r"\'ve", " have "), (r"can't", "cannot "), 
                   (r"n't", " not "), (r"i'm", "i am "), (r"\'re", " are "), (r"\'d", " would "), (r"\'ll", " will "), (r"\'scuse", " excuse ")]:
        text = re.sub(change[0], change[1], text)
    text = text.strip(' ')
    return text


train_df['comment_text'] = train_df['comment_text'].map(lambda com: clean_text(com))
test_df['comment_text'] = test_df['comment_text'].map(lambda com: clean_text(com))

x_train = train_df.comment_text
x_test = test_df.comment_text
print(x_train.shape, x_test.shape)

# vect = TfidfVectorizer(max_df=60000, min_df=2, stop_words='english')
vect = TfidfVectorizer(max_features=5000, stop_words='english')
# learn the vocabulary in the training data, then use it to create a document-term matrix
x_train_tfid = vect.fit_transform(x_train)
# transform the test data using the earlier fitted vocabulary, into a document-term matrix
x_test_tfid = vect.transform(x_test)



(159571,) (153164,)


In [26]:
logreg = LogisticRegression(C=12.0)

# create submission file
submission_binary = pd.DataFrame(index=test_df['id'], columns=cols_name)

for label in cols_name:
    print('... Processing {}'.format(label))
    y = train_df[label]
    yt = test_label[label]
    # train the model using X_dtm & y
    logreg.fit(x_train_tfid, y)
    # compute the training accuracy
    y_pred_X = logreg.predict(x_train_tfid)
    y_pred_t = logreg.predict(x_test_tfid)
    print('Training accuracy is {}'.format(accuracy_score(y, y_pred_X)))
    # compute the predicted probabilities for X_test_dtm
    test_y_prob = logreg.predict_proba(x_test_tfid)[:, 1]
    submission_binary[label] = test_y_prob
    print('Test accuracy is {}'.format(accuracy_score(yt, y_pred_t)))


... Processing obscene


Training accuracy is 0.9832300355327722
Test accuracy is 0.4023791491473192
... Processing insult


Training accuracy is 0.9755469352200588
Test accuracy is 0.4018503042490402
... Processing toxic


Training accuracy is 0.9639846839337975
Test accuracy is 0.3860894204904547
... Processing severe_toxic


Training accuracy is 0.9920787611784097
Test accuracy is 0.4145425818077355
... Processing identity_hate


Training accuracy is 0.9939713356436947
Test accuracy is 0.4136872894413831
... Processing threat


Training accuracy is 0.9981199591404453
Test accuracy is 0.4160050664647045


In [27]:
submission_binary


,obscene,insult,toxic,severe_toxic,identity_hate,threat
id,,,,,,
00001cee341fdb12,0.999965,0.971155,0.999956,0.463226,0.431323,0.050225
0000247867823ef7,0.000401,0.003284,0.002395,0.000431,0.000396,0.000329
00013b17ad220c46,0.003185,0.007276,0.011101,0.000119,0.001253,0.000035
00017563c3f7919a,0.000965,0.000873,0.001327,0.002131,0.000033,0.000117
00017695ad8997eb,0.001252,0.003282,0.019301,0.000834,0.000567,0.000607
0001ea8717f6de06,0.002695,0.010918,0.008125,0.000387,0.000369,0.000062
00024115d4cbde0f,0.000471,0.000840,0.000333,0.000313,0.000106,0.000017
000247e83dcc1211,0.044436,0.115899,0.674505,0.000323,0.000219,0.002996
00025358d4737918,0.009756,0.002035,0.002338,0.000475,0.000122,0.001705
